In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from torch.utils.data import TensorDataset, DataLoader

In [3]:
import lightning as pl
from lightning.pytorch.loggers import TensorBoardLogger
import torchmetrics

In [4]:
# config
# epochs = 80
max_epochs = 150
learning_rate = 0.0005

conv1_kernels = 136
conv2_kernels = 108

conv1_filter_size = 2
conv2_filter_size = 2

# in_features=1152
input_channels = 19 # like color channels one more than
channels = 19
h1 = 2000#1624
h2 = 1200#1624
h3 = 400#200
out_features = 128

batch_size = 512 # batch is about 0.5% of whole dataset, so each step will be 1/200 of an epoch

logging_name = 'CNN2'

project = "chess CNN model 2"

In [5]:
data_training = torch.load('tensor_data/nn2_rapid_games_data_training_09_02_24.pt')
data_training = data_training.view(data_training.shape[0], channels, 8, 8)
print(data_training.shape)
# data_training.to(device)

torch.Size([134183, 19, 8, 8])


In [6]:
data_training.dtype

torch.float64

In [7]:
data_testing = torch.load('tensor_data/nn2_rapid_games_data_testing_09_02_24.pt')
data_testing = data_testing.view(data_testing.shape[0], channels, 8, 8)
# print(data_testing.shape)
# data_testing.to(device)

In [8]:
lables_training = torch.load('tensor_data/nn2_rapid_games_lables_training_09_02_24.pt')
print(lables_training.shape)
# lables_training.to(device)

torch.Size([134183])


In [9]:
lables_testing = torch.load('tensor_data/nn2_rapid_games_lables_testing_09_02_24.pt')
print(lables_testing.shape)
# lables_testing.to(device)

torch.Size([33546])


In [11]:
data_training = data_training.float()
data_testing = data_testing.float()

In [12]:
lables_training = lables_training.long()
lables_testing = lables_testing.long()

In [13]:
torch.manual_seed(42)

In [14]:
training_dataset = TensorDataset(data_training, lables_training)
testing_dataset = TensorDataset(data_testing, lables_testing)

In [15]:
data_loader_training = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=7)
data_loader_testing = DataLoader(testing_dataset, batch_size=batch_size, num_workers=7)


In [16]:
class Cnn(pl.LightningModule):

    def __init__(self, input_channels=input_channels, h1=h1, h2=h2, h3=h3, out_features=out_features): #out = 64 squares to move to 

        super().__init__()

        self.conv1 = nn.Conv2d(input_channels, conv1_kernels, conv1_filter_size, 1, padding='same')
        self.conv2 = nn.Conv2d(conv1_kernels, conv2_kernels, conv2_filter_size, 1, padding='same')
        self.fc1 = nn.Linear(8*8*conv2_kernels, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, h3)
        # self.fc3 = nn.Linear(h2, h3)


        self.out = nn.Linear(h3, out_features)
        self.accuracy = torchmetrics.classification.Accuracy(task="multiclass", num_classes=out_features)

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))


        x = x.view(-1, 8*8*conv2_kernels)
        # x = x.view(-1,1152)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))

        x = self.out(x)

        return x

    def training_step(self, batch, batch_index):

        x, y = batch
        preds = self.forward(x) # since we run itself
        # print(preds)
        loss = F.cross_entropy(preds, y)
        
        accuracy = self.accuracy(preds, y)
        # self.log('train_acc_step', self.accuracy)
        self.log('loss', loss)
        self.log('accuracy', accuracy)

        return loss

    def validation_step(self, batch, batch_index):

        x, y = batch
        preds = self.forward(x) # since we run itself
        loss = F.cross_entropy(preds, y)
        accuracy = self.accuracy(preds, y)
        
        # self.log('train_acc_step', self.accuracy)
        self.log('loss validation', loss)
        self.log('accuracy validation', accuracy)

        return loss

    def configure_optimizers(self):

        return torch.optim.Adam(self.parameters(), lr=learning_rate)



In [17]:
model = Cnn()

In [18]:
logger = TensorBoardLogger("lightning_logs", name=logging_name) #tb logs is a directory?

In [19]:
# trainer = pl.Trainer(max_epochs=20, logger=logger, precision='32')
trainer = pl.Trainer(max_epochs=max_epochs, logger=logger, precision='16-mixed')

trainer.fit(model, data_loader_training, data_loader_testing)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type               | Params
------------------------------------------------
0 | conv1    | Conv2d             | 10.5 K
1 | conv2    | Conv2d             | 58.9 K
2 | fc1      | Linear             | 13.8 M
3 | fc2      | Linear             | 2.4 M 
4 | fc3      | Linear             | 480 K 
5 | out      | Linear             | 51.3 K
6 | accuracy | MulticlassAccuracy | 0     
------------------------------------------------
16.8 M    Trainable params
0         Non-trainable params
16.8 M    Total params
67.313    Total estimated model params size (MB)


/home/maru/miniforge3/envs/torch_chess/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1705951428005/work/aten/src/ATen/native/Convolution.cpp:1008.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch 98:  94%|█████████▍| 247/263 [00:07<00:00, 31.43it/s, v_num=4]

/home/maru/miniforge3/envs/torch_chess/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [20]:
torch.save(model.state_dict(), f'models/cnn2_ck1{conv1_kernels}_ck2{conv2_kernels}_fc1{h1}_fc2{h3}_e_{max_epochs}.pt')

In [21]:
# torch.save(Cnn, f'model_definitions/nn1_model_def_ck1{conv1_kernels}_ck2{conv2_kernels}_fc1{l1}_fc2{l2}_e_{epochs}.pt')

In [22]:
# torch.save(model, 'models/nn1_model_cnn_128_128_1624_1624_100_batches.pt')

In [23]:
# torch.save(model.state_dict(), 'models/nn1_model_linear_900_900.pt')